In [3]:
import findspark
findspark.init()

In [4]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
import torch
from random import Random
import pandas as pd
import csv

In [5]:
sc = SparkSession.builder.master("local").getOrCreate()
# sc = SparkSession.builder.master("spark://cheyenne:30915").getOrCreate()

Read the data into a pyspark dataframe.

In [6]:
northRegionInitialData = sc.read.csv("C:\\Users\Marky\\Documents\\College Schoolwork\\grad school work\\CSU\\cs 535 big data\\final project\\south.csv")

Rename and drop unneeded columns per our feature selection decisions.

In [53]:
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c0", "index")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c1", "date")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c2", "hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c3", "total_precipitation")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c4", "atmospheric_pressure_at_station_height")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c5", "pressure_max_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c6", "pressure_min_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c7", "radiation")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c8", "air_temperature")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c9", "dew_point_temperature")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c10", "max_temperature_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c11", "min_temperature_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c12", "max_dew_temperature_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c13", "min_dew_temperature_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c14", "humidity_max_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c15", "humidity_min_previous_hour")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c16", "air_relative_humidity_percentage")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c17", "wind_direction_meters_per_second")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c18", "wind_gust_meters_per_second")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c19", "wind_speed")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c20", "region")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c21", "state")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c22", "station")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c23", "station_code")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c24", "latitude")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c25", "longitude")
northRegionInitialData = northRegionInitialData.withColumnRenamed("_c26", "height")

In [54]:
northRegionInitialData = northRegionInitialData.filter("total_precipitation != -9999")
northRegionInitialData = northRegionInitialData.filter("atmospheric_pressure_at_station_height != -9999")
northRegionInitialData = northRegionInitialData.filter("pressure_max_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("pressure_min_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("radiation != -9999")
northRegionInitialData = northRegionInitialData.filter("air_temperature != -9999")
northRegionInitialData = northRegionInitialData.filter("dew_point_temperature != -9999")
northRegionInitialData = northRegionInitialData.filter("max_temperature_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("min_temperature_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("max_dew_temperature_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("min_dew_temperature_previous_hour != -9999")
northRegionInitialData = northRegionInitialData.filter("air_relative_humidity_percentage != -9999")
northRegionInitialData = northRegionInitialData.filter("wind_direction_meters_per_second != -9999")
northRegionInitialData = northRegionInitialData.filter("wind_gust_meters_per_second != -9999")
northRegionInitialData = northRegionInitialData.filter("wind_speed != -9999")

In [46]:
# northRegionInitialData.count()

3362209

In [55]:
northRegionInitialData = northRegionInitialData.drop("region", "state", "station", "station_code", "latitude", "longitude", "height")

In [56]:
northRegionInitialData.show(1)

+-----+----------+-----+-------------------+--------------------------------------+--------------------------+--------------------------+---------+---------------+---------------------+-----------------------------+-----------------------------+---------------------------------+---------------------------------+--------------------------+--------------------------+--------------------------------+--------------------------------+---------------------------+----------+
|index|      date| hour|total_precipitation|atmospheric_pressure_at_station_height|pressure_max_previous_hour|pressure_min_previous_hour|radiation|air_temperature|dew_point_temperature|max_temperature_previous_hour|min_temperature_previous_hour|max_dew_temperature_previous_hour|min_dew_temperature_previous_hour|humidity_max_previous_hour|humidity_min_previous_hour|air_relative_humidity_percentage|wind_direction_meters_per_second|wind_gust_meters_per_second|wind_speed|
+-----+----------+-----+-------------------+----------

In [57]:
northRegionInitialDataUnEdited = northRegionInitialData

This weather dataset starts at 2000-05. There is no data from 1999, but there is data up to and including 2021-04. There is no data in 2022.

Below is the main function which formats the data into the way we want it. We loop over all years with data, 2000-2021, and average out all values by month and year. This effectively means we are taking averages of each of the values over individual months. This will leave us with 12 months * 22 years = 264 data points for each region. There are five regions. 

In [68]:
averagesInListFormat2000To2021 = []
year = 2000
while year < 2022:
    for month in range(9):
        month+=1
        northRegionInitialData = northRegionInitialDataUnEdited
        northRegionInitialData = northRegionInitialData.filter(northRegionInitialData.date.contains(str(year) + '-0' + str(month) + '-'))
        averagesInListFormat2000To2021.append(northRegionInitialData.agg({'total_precipitation': 'avg', 'atmospheric_pressure_at_station_height': 'avg', 'pressure_max_previous_hour': 'avg', 'pressure_min_previous_hour': 'avg', 'radiation': 'avg', 'air_temperature': 'avg', 'dew_point_temperature': 'avg', 'max_temperature_previous_hour': 'avg', 'min_temperature_previous_hour': 'avg', 'max_dew_temperature_previous_hour': 'avg', 'min_dew_temperature_previous_hour': 'avg', 'humidity_max_previous_hour': 'avg', 'humidity_min_previous_hour': 'avg', 'air_relative_humidity_percentage': 'avg', 'wind_direction_meters_per_second': 'avg', 'wind_gust_meters_per_second': 'avg', 'wind_speed': 'avg'}).collect()[0])

    northRegionInitialData = northRegionInitialDataUnEdited
    northRegionInitialData = northRegionInitialData.filter(northRegionInitialData.date.contains(str(year) + '-10' + '-'))
    averagesInListFormat2000To2021.append(northRegionInitialData.agg({'total_precipitation': 'avg', 'atmospheric_pressure_at_station_height': 'avg', 'pressure_max_previous_hour': 'avg', 'pressure_min_previous_hour': 'avg', 'radiation': 'avg', 'air_temperature': 'avg', 'dew_point_temperature': 'avg', 'max_temperature_previous_hour': 'avg', 'min_temperature_previous_hour': 'avg', 'max_dew_temperature_previous_hour': 'avg', 'min_dew_temperature_previous_hour': 'avg', 'humidity_max_previous_hour': 'avg', 'humidity_min_previous_hour': 'avg', 'air_relative_humidity_percentage': 'avg', 'wind_direction_meters_per_second': 'avg', 'wind_gust_meters_per_second': 'avg', 'wind_speed': 'avg'}).collect()[0])        
    
    northRegionInitialData = northRegionInitialDataUnEdited
    northRegionInitialData = northRegionInitialData.filter(northRegionInitialData.date.contains(str(year) + '-11' + '-'))
    averagesInListFormat2000To2021.append(northRegionInitialData.agg({'total_precipitation': 'avg', 'atmospheric_pressure_at_station_height': 'avg', 'pressure_max_previous_hour': 'avg', 'pressure_min_previous_hour': 'avg', 'radiation': 'avg', 'air_temperature': 'avg', 'dew_point_temperature': 'avg', 'max_temperature_previous_hour': 'avg', 'min_temperature_previous_hour': 'avg', 'max_dew_temperature_previous_hour': 'avg', 'min_dew_temperature_previous_hour': 'avg', 'humidity_max_previous_hour': 'avg', 'humidity_min_previous_hour': 'avg', 'air_relative_humidity_percentage': 'avg', 'wind_direction_meters_per_second': 'avg', 'wind_gust_meters_per_second': 'avg', 'wind_speed': 'avg'}).collect()[0])
        
        
    northRegionInitialData = northRegionInitialDataUnEdited
    northRegionInitialData = northRegionInitialData.filter(northRegionInitialData.date.contains(str(year) + '-12' + '-'))
    averagesInListFormat2000To2021.append(northRegionInitialData.agg({'total_precipitation': 'avg', 'atmospheric_pressure_at_station_height': 'avg', 'pressure_max_previous_hour': 'avg', 'pressure_min_previous_hour': 'avg', 'radiation': 'avg', 'air_temperature': 'avg', 'dew_point_temperature': 'avg', 'max_temperature_previous_hour': 'avg', 'min_temperature_previous_hour': 'avg', 'max_dew_temperature_previous_hour': 'avg', 'min_dew_temperature_previous_hour': 'avg', 'humidity_max_previous_hour': 'avg', 'humidity_min_previous_hour': 'avg', 'air_relative_humidity_percentage': 'avg', 'wind_direction_meters_per_second': 'avg', 'wind_gust_meters_per_second': 'avg', 'wind_speed': 'avg'}).collect()[0])
    
    year+=1

In [69]:
print(len(averagesInListFormat2000To2021))

264


In [72]:
averagesInListFormat2000To2021[255]

Row(avg(dew_point_temperature)=22.747106705351367, avg(wind_direction_meters_per_second)=166.1942295293359, avg(radiation)=1071.5106382978724, avg(min_temperature_previous_hour)=26.35620567375887, avg(atmospheric_pressure_at_station_height)=992.4186331399077, avg(pressure_min_previous_hour)=992.0095099935495, avg(total_precipitation)=0.3196808510638284, avg(max_dew_temperature_previous_hour)=23.347558027079298, avg(wind_gust_meters_per_second)=3.6982914248871737, avg(air_temperature)=27.09333494519663, avg(wind_speed)=1.2781753707285635, avg(max_temperature_previous_hour)=27.734840425531875, avg(humidity_min_previous_hour)=75.29867827208253, avg(pressure_max_previous_hour)=992.9063265635054, avg(humidity_max_previous_hour)=82.35726950354609, avg(air_relative_humidity_percentage)=78.65385235332045, avg(min_dew_temperature_previous_hour)=22.154529335912336)

Now we write out the data to a csv file so we can combine them all. We do this for each region: north, south, west, southeast, and northeast.

In [ ]:
header = ['time_stamp', 'dew_point_temperature', 'wind_direction_meters_per_second', 'radiation', 'min_temperature_previous_hour', 'atmospheric_pressure_at_station_height', 'pressure_min_previous_hour','max_dew_temperature_previous_hour','wind_gust_meters_per_second','air_temperature','wind_speed','max_temperature_previous_hour','humidity_min_previous_hour','pressure_max_previous_hour','humidity_max_previous_hour','air_relative_humidity_percentage','min_dew_temperature_previous_hour','total_precipitation']

with open('southRegionAllFeatures.csv', 'w', newline='') as file:
    write = csv.writer(file)
    write.writerow(header)
    i = 0
    for row in averagesInListFormat2000To2021:
        write.writerow([timestamp[i], row[0], row[1], row[2], row[3], row[4], row[5], row[7], row[8],row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[6]])
        i = i + 1

In [43]:
with open('southRegionAllFeatures.csv', newline='') as readfile:
    reader = csv.reader(readfile)
    data = tuple(reader)